## Data processing

This Jupyter Noterbook helps us to convert binary attribute(s) to +/-1, categorical attributes(s) to onehot.

In [1]:
import numpy as np
from sklearn.preprocessing import OneHotEncoder

We load the data which were cleaned from the `data cleaning` step.

In [2]:
Xy = np.loadtxt('smoking_cleaned.dat', dtype = 'str')

print(Xy.shape)
print(Xy)

(1574, 96)
[['35.0' '2.0' '1.0' ... '1.0' '1.0' '0.0']
 ['34.0' '1.0' '4.0' ... '1.0' '1.0' '0.0']
 ['37.0' '2.0' '3.0' ... '3.0' '1.0' '0.0']
 ...
 ['28.0' '1.0' '4.0' ... '1.0' '4.0' '0.0']
 ['29.0' '1.0' '2.0' ... '3.0' '2.0' '0.0']
 ['47.0' '2.0' '1.0' ... '2.0' '1.0' '0.0']]


### Attributes

We find number of unique value for each column, to have an idea about which variables are continuous, which variables are binary, category. It depends on data, however it is likely that nu = 2 --> binary; nu = 3 or 4: --> category, n > 4: continuous. Of course, we have to see data in detail as well.

In [3]:
X = Xy[:,:-1]
l,n = X.shape
nu = np.array([len(np.unique(X[:,i])) for i in range(n)])
print('number of uniques of each variable:')
print(nu)

number of uniques of each variable:
[  58    2    5    3    5    4    6    3    3    2    2    2    3    4
    3    3    3    6    3    3    3    4    3    3    3    3    3    3
    8    9    3    3    3    3    3    3    3    4    4    4    4    3
 1565   10    4    3    3    3    5    2    3    7    3    2    2    2
    2    3    3    3    3    3    4    2    2    2    2    2    2    2
    2    9    2    9    2    2    2    2    2    2    2    2    2    2
    2    2    2    2    2    2    2    2    2    4    4]


In [4]:
i_binary = []
i_category = []
i_continuous = []
for i in range(X.shape[1]):
    nu = np.unique(X[:,i])
    if len(nu) == 2: # binary 
        i_binary.append(i)
    elif len(nu) < 5:
        i_category.append(i)
    else:
        i_continuous.append(i)
        
print('i_binary:',i_binary)
print('i_category:',i_category)

i_binary: [1, 9, 10, 11, 49, 53, 54, 55, 56, 63, 64, 65, 66, 67, 68, 69, 70, 72, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92]
i_category: [3, 5, 7, 8, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 44, 45, 46, 47, 50, 52, 57, 58, 59, 60, 61, 62, 93, 94]


We then define variable type, 1: continuous, 2: binary, 3: category.

In [5]:
variable_type  = np.ones(n) # continuous
variable_type[i_binary] = 2 # binary
variable_type[i_category] = 3 # categorical

print(variable_type)

[1. 2. 1. 3. 1. 3. 1. 3. 3. 2. 2. 2. 3. 3. 3. 3. 3. 1. 3. 3. 3. 3. 3. 3.
 3. 3. 3. 3. 1. 1. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 3. 1. 1. 3. 3. 3. 3.
 1. 2. 3. 1. 3. 2. 2. 2. 2. 3. 3. 3. 3. 3. 3. 2. 2. 2. 2. 2. 2. 2. 2. 1.
 2. 1. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 2. 3. 3.]


We now convert binary to +/-1, category to onehot.

In [6]:
def convert_binary_and_category(x,variable_type):
    """
    convert binary to +-1, category to one hot; remain continuous.
    """
    
    onehot_encoder = OneHotEncoder(sparse=False,categories='auto')

    # create 2 initial columns
    x_new = np.zeros((x.shape[0],2))

    for i,i_type in enumerate(variable_type):
        if i_type == 1: # continuous
            x_new = np.hstack((x_new,x[:,i][:,np.newaxis]))
        elif i_type == 2: # binary
            unique_value = np.unique(x[:,i])
            x1 = np.array([-1. if value == unique_value[0] else 1. for value in x[:,i]])        
            x_new = np.hstack((x_new,x1[:,np.newaxis]))
        else: # category      
            x1 = onehot_encoder.fit_transform(x[:,i].reshape(-1,1))
            x_new = np.hstack((x_new,x1))        

    # drop the 2 initial column
    x_new = x_new[:,2:]
    
    return x_new.astype(float)

In [7]:
# convert X
X_new = convert_binary_and_category(X,variable_type)

print(X_new.shape)
print(X_new)

(1574, 196)
[[35.  1.  1. ...  0.  0.  0.]
 [34. -1.  4. ...  0.  0.  0.]
 [37.  1.  3. ...  0.  0.  0.]
 ...
 [28. -1.  4. ...  0.  0.  1.]
 [29. -1.  2. ...  1.  0.  0.]
 [47.  1.  1. ...  0.  0.  0.]]


### Target

In [8]:
## target
y = Xy[:,-1].astype(float)

print(np.unique(y,return_counts=True))

(array([0., 1.]), array([852, 722]))


In [9]:
y_new = y
# convert taget to 0 and 1
#y_new = np.ones(y.shape[0])


print(np.unique(y_new,return_counts=True))

(array([0., 1.]), array([852, 722]))


In [10]:
# combine X and y
Xy_new = np.hstack((X_new,y_new[:,np.newaxis]))

In [11]:
np.savetxt('smoking_processed.dat',Xy_new,fmt='%f')